In [13]:
import pandas as pd

# Fájl beolvasása
file_path = r"C:\Users\rotar\OneDrive\Asztali gép\Cubix_Data_Challenge\SQL\MovieMasters\movie_master_clean.csv"
  # Adj meg helyes elérési utat
df = pd.read_csv(file_path, sep=';')

# Oszlopnevek megtekintése
print(df.columns)


Index(['film_id', 'imdb_id', 'title', 'duration_minutes', 'year_released',
       'genres', 'popularity', 'average_rating', 'production_cost',
       'box_office_revenue', 'actor1', 'actor2', 'actor3', 'director',
       'writer', 'production_company'],
      dtype='object')


In [22]:
import pandas as pd

# CSV betöltése
file_path = r"C:\Users\rotar\OneDrive\Asztali gép\Cubix_Data_Challenge\SQL\MovieMasters\movie_master_clean.csv"

# Frissítsd az elérési útvonalat, ha szükséges
df = pd.read_csv(file_path, sep=';')

# Ellenőrzés: vannak-e hiányzó értékek
df = df.dropna(subset=["writer", "director", "actor1", "actor2", "actor3", "average_rating", "popularity"])

# Oszlopok számértékké alakítása
df["average_rating"] = pd.to_numeric(df["average_rating"], errors="coerce")
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")

# Nem numerikus értékek (pl. hibás adatok) átalakítása NaN-ra
df["average_rating"].fillna(df["average_rating"].mean(), inplace=True)
df["popularity"].fillna(df["popularity"].mean(), inplace=True)

# Oszlopok szétválasztása és kibővítése
def explode_column(df, column_name):
    return df.assign(**{column_name: df[column_name].str.split(", ")}).explode(column_name)

# Írók
writers_df = explode_column(df[["writer", "average_rating", "popularity"]], "writer")
top_writers = writers_df.groupby("writer").agg({"average_rating": "mean", "popularity": "mean"}).sort_values(by=["average_rating", "popularity"], ascending=False).head(3)

# Rendezők
directors_df = explode_column(df[["director", "average_rating", "popularity"]], "director")
top_directors = directors_df.groupby("director").agg({"average_rating": "mean", "popularity": "mean"}).sort_values(by=["average_rating", "popularity"], ascending=False).head(3)

# Színészek: A három oszlopot először külön-külön kezeljük, majd összefűzzük őket
actor1_df = explode_column(df[["actor1", "average_rating", "popularity"]], "actor1")
actor2_df = explode_column(df[["actor2", "average_rating", "popularity"]], "actor2")
actor3_df = explode_column(df[["actor3", "average_rating", "popularity"]], "actor3")

# Az összes színész egyesítése
actors_combined_df = pd.concat([actor1_df[["actor1", "average_rating", "popularity"]],
                                actor2_df[["actor2", "average_rating", "popularity"]],
                                actor3_df[["actor3", "average_rating", "popularity"]]], ignore_index=True)

# Színészek csoportosítása
actors_combined_df = actors_combined_df.rename(columns={"actor1": "actor"}).fillna("")
top_actors = actors_combined_df.groupby("actor").agg({"average_rating": "mean", "popularity": "mean"}).sort_values(by=["average_rating", "popularity"], ascending=False).head(3)

# Eredmények kiírása
print("Top 3 író:")
print(top_writers)

print("\nTop 3 rendező:")
print(top_directors)

print("\nTop 3 főszereplő:")
print(top_actors)

# Save updated files
df.to_csv("movies_updated_cleaned_top3.csv", index=False)

print("Files saved successfully.")

Top 3 író:
              average_rating  popularity
writer                                  
Graham Moore             8.0    8.185185
Jun Lana                 8.0    8.185185
Luke Davies              8.0    8.185185

Top 3 rendező:
                  average_rating  popularity
director                                    
Garth Davis                  8.0    8.185185
Jun Lana                     8.0    8.185185
Mahesh Manjrekar             8.0    8.185185

Top 3 főszereplő:
                    average_rating  popularity
actor                                         
Bharat Jadhav                  8.0    8.185185
Eddie Garcia                   8.0    8.185185
Ferdia Walsh-Peelo             8.0    8.185185
Files saved successfully.


C:\Users\rotar\AppData\Local\Temp\ipykernel_5088\112693337.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["average_rating"].fillna(df["average_rating"].mean(), inplace=True)
C:\Users\rotar\AppData\Local\Temp\ipykernel_5088\112693337.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves